In [1]:
import tensorflow as tf
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import mplfinance as mpf
import talib as talib
import numpy as np
import data as ds
import common as common
import os as os
import math as math
import datetime as dt
import scipy as sp
import itertools  as itertools
import multiprocessing as mp
import joblib
from os import listdir, walk
from itertools import repeat
from mplfinance.original_flavor import candlestick_ohlc
from datetime import datetime, timedelta, date
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, pairwise, mean_squared_error, mean_absolute_error
from scipy import stats
from pprint import pprint

In [2]:
# 0.1 环境设定
pd.options.mode.chained_assignment = None

# 0.2 不让程序占满 GPU 内存
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
# 1.0 重構數據集
def reshape_dataframe(df0, year0):
    df0.fillna(0, inplace=True)
    df0.replace([np.inf, -np.inf], 0, inplace=True)
    df0.columns = ['udate', 'High', 'Low', 'Open', 'Close', 'Volume']
    types2 = {'udate': 'object', 'High': 'float64', 'Low': 'float64', 'Open': 'float64', 'Close': 'float64', 'Volume': 'int64'}
    df0.astype(types2).dtypes
    df0_1 = df0.copy(deep=True)
    error_row = []
    for k, v in df0.iterrows():
        if not pd.isnull(df0['udate'].iloc[k]) and df0['udate'].iloc[k] > 0:
            stime = str(int(df0['udate'].iloc[k]))
            df0_1['udate'].iloc[k] = datetime(year=year0, month=int(stime[-8:-6]), day=int(stime[-6:-4]), hour=int(stime[-4:-2]), minute=int(stime[-2:]), second=0)
        else:
            error_row.append(k)
    df0_1.drop(df0_1.index[error_row], inplace=True)
    df0_1.udate = pd.to_datetime(df0_1.udate)
    df0_1.index = pd.to_datetime(df0_1.udate)
    
    # 1.0.1 數據有效性檢查
    for k, v in types2.items():
        if (df0_1[k].isin([np.nan]).any().any()):
            print(k+' obtains nan')
        if (df0_1[k].isin([0]).any().any()):
            print(k+' obtains 0')
    is_contain_null = df0_1.isnull().sum()

    return df0_1

# 1.2.1 year 2020
path_files_1 = [os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201105.csv')),
                os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201105_Aug_Sep.csv')),
                os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201110.csv')),
                os.path.abspath(os.path.join('data', 'nq', 'data', 'nq-20201111.csv'))]
path_files_2 = []
for j in range(1, 13):
    month1 = str(j).zfill(2)
    file1 = 'nq-20-'+month1+'.csv'
    path_files_1.append(os.path.abspath(os.path.join('data', 'nq', 'data', file1)))
    file2 = 'nq-19-'+month1+'.csv'
    path_files_2.append(os.path.abspath(os.path.join('data', 'nq', 'data', file2)))
    
# 1.3
def data_worker1(m_list1_2, path1_2, year_2):
    try:
        df1_2 = reshape_dataframe(pd.read_csv(path1_2), year_2)
        m_list1_2.append(df1_2)
    except:
        print('No file', path1_2)

# 1.4
manager1 = mp.Manager()
m_list1 = manager1.list()
pool1 = mp.Pool(processes=2, maxtasksperchild=2)
for path1 in path_files_1:
    pool1.apply_async(func=data_worker1, args=(m_list1, path1, 2020,))
for path2 in path_files_2:
    pool1.apply_async(func=data_worker1, args=(m_list1, path2, 2019,))
pool1.close()
pool1.join()

Volume obtains 0
Volume obtains 0
Volume obtains 0
No file /home/user/PycharmProjects/JupyterLab01/data/nq/data/nq-20-01.csv
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
No file /home/user/PycharmProjects/JupyterLab01/data/nq/data/nq-20-10.csv
Volume obtains 0
No file /home/user/PycharmProjects/JupyterLab01/data/nq/data/nq-20-12.csv
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0
Volume obtains 0


In [4]:
# 1.5
df2 = pd.concat(m_list1, ignore_index=False)

# 1.6 刪除重覆index
df2 = df2.groupby(df2.index).first()

# 1.7 排序
df2.sort_index(axis=0, ascending=True, inplace=True)
# df2 = df2.loc[(df2.udate >= datetime(2019, 3, 1, 16, 0, 0))]
df2

,udate,High,Low,Open,Close,Volume
udate,,,,,,
2019-01-13 17:00:00,2019-01-13 17:00:00,6599.00,6602.50,6590.25,6595.75,567.0
2019-01-13 17:01:00,2019-01-13 17:01:00,6595.75,6598.75,6591.50,6595.75,269.0
2019-01-13 17:02:00,2019-01-13 17:02:00,6595.75,6596.75,6589.00,6591.00,225.0
2019-01-13 17:03:00,2019-01-13 17:03:00,6591.00,6592.75,6588.00,6588.25,142.0
2019-01-13 17:04:00,2019-01-13 17:04:00,6588.25,6590.25,6586.50,6588.75,214.0
...,...,...,...,...,...,...
2020-11-19 02:25:00,2020-11-19 02:25:00,11864.25,11871.50,11864.25,11869.25,155.0
2020-11-19 02:26:00,2020-11-19 02:26:00,11869.25,11871.25,11868.00,11868.50,114.0
2020-11-19 02:27:00,2020-11-19 02:27:00,11868.50,11872.25,11868.25,11870.50,85.0


In [5]:
# 2.0 分包 (日子)
def separate_daily(df2_1, return_type):
    data1 = {}
    # 2.1交易日
    days1 = list(dict.fromkeys([v.date() for v in df2_1['udate']]))
    for day in days1:
        # 2.2 交易時間
        day_start = datetime(day.year, day.month, day.day, 17, 0, 0)
        day2 = day + timedelta(days=1)
        day_end = datetime(day2.year, day2.month, day2.day, 16, 0, 0)
        mask = ((df2_1['udate'] >= day_start) & (df2_1['udate'] <= day_end))
        df2_2 = df2_1.loc[mask]
        if (df2_2.shape[0] > 1):
            data1[day] = df2_2
    # 2.3 合併
    df2_3 = pd.DataFrame()
    for k, df2_4 in data1.items():
        df2_3 = pd.concat([df2_3, df2_4], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
    # 2.4 返回類型
    if return_type=='df':
        return df2_3
    elif return_type=='dict':
        return data1

df2_5 = df2.copy(deep=True)
df3 = separate_daily(df2_5, 'df')
df3

,udate,High,Low,Open,Close,Volume
udate,,,,,,
2019-01-13 17:00:00,2019-01-13 17:00:00,6599.00,6602.50,6590.25,6595.75,567.0
2019-01-13 17:01:00,2019-01-13 17:01:00,6595.75,6598.75,6591.50,6595.75,269.0
2019-01-13 17:02:00,2019-01-13 17:02:00,6595.75,6596.75,6589.00,6591.00,225.0
2019-01-13 17:03:00,2019-01-13 17:03:00,6591.00,6592.75,6588.00,6588.25,142.0
2019-01-13 17:04:00,2019-01-13 17:04:00,6588.25,6590.25,6586.50,6588.75,214.0
...,...,...,...,...,...,...
2020-11-19 02:25:00,2020-11-19 02:25:00,11864.25,11871.50,11864.25,11869.25,155.0
2020-11-19 02:26:00,2020-11-19 02:26:00,11869.25,11871.25,11868.00,11868.50,114.0
2020-11-19 02:27:00,2020-11-19 02:27:00,11868.50,11872.25,11868.25,11870.50,85.0


In [6]:
# 3.0 技術指標
highs = np.array(df3['High'], dtype='float')
lows = np.array(df3['Low'], dtype='float')
opens = np.array(df3['Open'], dtype='float')
closes = np.array(df3['Close'], dtype='float')
vols = np.array(df3['Volume'], dtype='float')
# 3.1 Bollinger 保力加
df3['upper-band'], df3['middle-band'], df3['lower-band'] = talib.BBANDS(closes, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
# 3.2 %B %保力加
df3['%b'] = (df3['Close']-df3['lower-band'])/(df3['upper-band']-df3['lower-band'])*100
df3['%b-high']  = common.percentB_belowzero(df3['%b'], df3['Close']) 
df3['%b-low'] = common.percentB_aboveone(df3['%b'], df3['Close'])
# 2.3 MACD
weight = 1.5
df3['macd'], df3['macdsignal'], df3['macdhist'] = talib.MACD(closes, fastperiod=12*weight, slowperiod=26*weight, signalperiod=9*weight)
# 2.4 RSI
df3['rsi-2'] = talib.RSI(closes, timeperiod=14*weight)
df3['rsi'] = df3['rsi-2']
df3['rsi'].loc[((df3['rsi'] < 85) & (df3['rsi'] > 25))] = 0
# 2.5 KDJ
df3['k-kdj'], df3['d-kdj'], df3['j-kdj'] = common.kdj(highs, lows, closes, window_size=5)
df3['diff-kdj'] = df3['k-kdj']-df3['d-kdj']
df3['j-kdj'].loc[((df3['j-kdj'] > 20) & (df3['j-kdj'] < 100))] = 0
# 3.6 VWAP 成交量加權平均價格
period = []
for v in period:
    df3['typical-price'] = (df3['High'] + df3['Low'] + df3['Close']) / 3
    df3['turnover'] = df3['typical-price'] * df3['Volume']
    df3['cum-turnover-'+str(v)] = df3['turnover'].rolling(window=v).sum()
    df3['cum-volume-'+str(v)] = df3['Volume'].rolling(window=v).sum()
    df3['vwap-'+str(v)] = df3['cum-turnover-'+str(v)] / df3['cum-volume-'+str(v)]
    df3['vwap-'+str(v)] = df3['vwap-'+str(v)].replace([np.inf, -np.inf], 0)
    df3['vwap-'+str(v)].fillna(0, inplace=True)
    drop_list_1 = ['turnover', 'typical-price', 'cum-turnover-'+str(v), 'cum-volume-'+str(v)]
    df3.drop(drop_list_1, axis=1, inplace=True)
# 3.7 SMA 均線
for v in [15, 50, 100, 200]:
    df3['sma-'+str(v)]= talib.SMA(closes, timeperiod=v)
# 3.8 VOL EMA
df3['vol-ema5'] = talib.EMA(vols, timeperiod=5)
# 3.9 P-SAR 抛物线
#df3['p-sar'] = talib.SAR(highs, lows, acceleration=0.02, maximum=0.2)
df3

,udate,High,Low,Open,Close,Volume,upper-band,middle-band,lower-band,%b,...,rsi,k-kdj,d-kdj,j-kdj,diff-kdj,sma-15,sma-50,sma-100,sma-200,vol-ema5
udate,,,,,,,,,,,,,,,,,,,,,
2019-01-13 17:00:00,2019-01-13 17:00:00,6599.00,6602.50,6590.25,6595.75,567.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-13 17:01:00,2019-01-13 17:01:00,6595.75,6598.75,6591.50,6595.75,269.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-13 17:02:00,2019-01-13 17:02:00,6595.75,6596.75,6589.00,6591.00,225.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-13 17:03:00,2019-01-13 17:03:00,6591.00,6592.75,6588.00,6588.25,142.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-13 17:04:00,2019-01-13 17:04:00,6588.25,6590.25,6586.50,6588.75,214.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-19 02:25:00,2020-11-19 02:25:00,11864.25,11871.50,11864.25,11869.25,155.0,11874.155992,11863.1625,11852.169008,77.686834,...,0.0,9.187728,-6.309339,0.0,15.497067,11861.100000,11871.445,11879.9425,11876.64375,102.712294
2020-11-19 02:26:00,2020-11-19 02:26:00,11869.25,11871.25,11868.00,11868.50,114.0,11873.673826,11863.0250,11852.376174,75.707059,...,0.0,12.651780,-5.427407,0.0,18.079187,11861.250000,11871.220,11879.7725,11876.69750,106.474863
2020-11-19 02:27:00,2020-11-19 02:27:00,11868.50,11872.25,11868.25,11870.50,85.0,11873.189021,11862.9125,11852.635979,86.916678,...,0.0,17.416898,-4.204812,0.0,21.621709,11861.583333,11871.160,11879.6075,11876.76500,99.316575


In [7]:
# 4.0 draw chart
drop_list_2 = []
is_render_chart = False

def draw_worker(df4_2, day):
    try:
        # 4.1 style
        style = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size':6})
        # 4.2 addplot
        apds = [mpf.make_addplot(df4_2['lower-band'],panel=0,color='orange',linestyle='solid'),
                mpf.make_addplot(df4_2['upper-band'],panel=0,color='bisque',linestyle='solid'),
                # mpf.make_addplot(df4_2['vwap-50'].replace(0, np.nan),panel=0,color='aqua',linestyle='solid'),
                mpf.make_addplot(df4_2['%b-low'],type='scatter',markersize=20,marker='v',panel=0),
                mpf.make_addplot(df4_2['%b-high'],type='scatter',markersize=20,marker='^',panel=0),
                # mpf.make_addplot(df4_2['p-sar'],scatter=True,markersize=1,marker='*',panel=0,color='blueviolet'),
                #
                mpf.make_addplot(df4_2['vol-ema5'],panel=1,color='orange'),
                #
                mpf.make_addplot(df4_2['macd'],panel=2,color='orange'),
                mpf.make_addplot(df4_2['macdsignal'],panel=2,color='violet'),
                mpf.make_addplot(df4_2['macdhist'],panel=2,type='bar',color='dimgray'),
                #
                mpf.make_addplot(df4_2['k-kdj'],panel=3,color='orange'),
                mpf.make_addplot(df4_2['d-kdj'],panel=3,color='violet'),
                mpf.make_addplot(df4_2['j-kdj'],panel=3,color='aqua'),
                mpf.make_addplot(df4_2['diff-kdj'],panel=3,type='bar',color='dimgray'),
                #
                mpf.make_addplot(df4_2['rsi-2'],panel=4,color='orange'),
                mpf.make_addplot(df4_2['rsi'],panel=4,color='violet')]
        # 4.3 draw
        mpf.plot(df4_2, type='candle', addplot=apds, style=style, ylabel='', ylabel_lower='', volume=True, figscale=0.85, xrotation=0, datetime_format="%H:%M", 
                 show_nontrading=False, tight_layout=True, savefig='./data/img-nq/features/'+day.strftime('%Y-%m-%d'))
        print('finish draw:', df4_2.shape, df4_2['udate'].iloc[0], df4_2['udate'].iloc[-1])
    except:
        print('do not draw:', k, '\n')

# 4.5 multi processing
pool = mp.Pool(processes=4, maxtasksperchild=4)
# 4.6 draw
df4 = df3.copy(deep=True)
data2 = separate_daily(df4, 'dict')
for k, df4_1 in data2.items():
    # 4.7 drop error day
    if df4_1['Volume'].shape[0] == df4_1['Volume'].isin([0]).sum() or df4_1['Volume'].shape[0] <= 100:
        drop_list_2.append(k)
    # 4.8 draw chart
    elif (is_render_chart):
        pool.apply_async(draw_worker, args=(df4_1, k))
    if k.weekday() == 6:
        drop_list_2.append(k)
# 4.9 kill multi processing
pool.close()
pool.join()

drop_list_2.append(date(2019, 1, 13))
drop_list_2.append(date(2019, 1, 20))
drop_list_2.append(date(2019, 3, 8))
drop_list_2.append(date(2019, 3, 15))
drop_list_2.append(date(2019, 3, 16))
drop_list_2.append(date(2019, 3, 17))
drop_list_2.append(date(2019, 5, 24))
drop_list_2.append(date(2019, 5, 25))
drop_list_2.append(date(2019, 5, 26))

drop_list_2.append(date(2020, 3, 8))
drop_list_2.append(date(2020, 3, 15))
drop_list_2.append(date(2020, 3, 17))
drop_list_2.append(date(2020, 5, 24))
drop_list_2.append(date(2020, 9, 6))
for k in drop_list_2:
    data2.pop(k, None)
    print('excpet & delete: ', k)

excpet & delete:  2019-01-13
excpet & delete:  2019-01-20
excpet & delete:  2019-01-27
excpet & delete:  2019-02-10
excpet & delete:  2019-02-17
excpet & delete:  2019-02-24
excpet & delete:  2019-03-03
excpet & delete:  2019-03-10
excpet & delete:  2019-03-17
excpet & delete:  2019-03-24
excpet & delete:  2019-04-21
excpet & delete:  2019-04-28
excpet & delete:  2019-05-12
excpet & delete:  2019-05-19
excpet & delete:  2019-05-26
excpet & delete:  2019-06-02
excpet & delete:  2019-06-09
excpet & delete:  2019-06-16
excpet & delete:  2019-06-23
excpet & delete:  2019-07-21
excpet & delete:  2019-07-28
excpet & delete:  2019-08-04
excpet & delete:  2019-08-11
excpet & delete:  2019-08-18
excpet & delete:  2019-08-25
excpet & delete:  2019-09-08
excpet & delete:  2019-09-15
excpet & delete:  2019-09-22
excpet & delete:  2019-09-29
excpet & delete:  2019-10-13
excpet & delete:  2019-10-27
excpet & delete:  2019-11-10
excpet & delete:  2019-11-17
excpet & delete:  2019-11-24
excpet & delet

In [8]:
# 5.0 合併
df5 = pd.DataFrame()
for k, df5_1 in data2.items():
    df5 = pd.concat([df5, df5_1], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

# 5.1 清洗
df5_3 = df5.copy(deep=True)
drop_list_3 = ['High', 'Low', 'Open', 'Volume', 'macd', 'macdsignal', 'upper-band', 'lower-band', 'middle-band', 'k-kdj', 'd-kdj', 'vol-ema5', 'rsi-2']
df5_3.drop(drop_list_3, axis=1, inplace=True)
df5_3.fillna(0, inplace=True)
df5_3 = df5_3.round(2)

# 5.2 檢查
is_contain_null = df5_3.isnull().sum()
is_contain_nan = df5_3.isna().sum()
is_contain_inf = df5_3.isin([np.nan]).sum()
print('数据集: Total dataset has {} samples, and {} features.'.format(df5_3.shape[0], df5_3.shape[1])) # df5_3.info()

# 5.3 儲存
path_data = os.path.abspath(os.path.join('data', 'nq', 'clean-data', 'nq-clean-data-with-features.csv'))
if os.path.exists(path_data):
    os.remove(path_data)
df5_3.to_csv(path_data)

# 5.4.1 分包 (日子)
data3 = separate_daily(df5_3, 'dict')
# 5.3.2 正則化
normalization_days = 10
days2 = np.array(list(data3.keys()))
len_days2 = (days2.shape[0]//normalization_days)+1
data3_1 = []
scalers = {}
for i in range(len_days2):
    try:
        start1, end1 = i*normalization_days, (i+1)*normalization_days
        if end1 >= days2.shape[0]:
            end1 = days2.shape[0]-1
        # 5.3.3 每X日集合
        day_start, day_end = days2[start1], days2[end1]
        day_start2 = datetime(day_start.year, day_start.month, day_start.day, 17, 0, 0)
        day_end2 = datetime(day_end.year, day_end.month, day_end.day, 16, 0, 0)
        mask = ((df5_3['udate'] >= day_start2) & (df5_3['udate'] <= day_end2))
        df5_4 = df5_3[mask]
        # 5.4.4 正則代集合
        min_max_scaler = MinMaxScaler(feature_range=(0, 1))
        df5_5 = df5_4.drop(['udate'], axis=1)
        min_max_scaler.fit(df5_5)
        scalers[i] = {'scaler': min_max_scaler, 'day_start': day_start2, 'day_end': day_end2}
        df5_6 = min_max_scaler.transform(df5_5)
        df5_6 = pd.DataFrame(df5_6, columns=df5_5.columns, index=df5_5.index)
        df5_6['udate'] = df5_4['udate']
        # 5.4.5 合併
        data3_1.append(df5_6)
    except:
        print('except: ', start1, end1, len(days2))

# 5.4.6 合併
df5_7 = pd.DataFrame()
for df5_8 in data3_1:
    df5_7 = pd.concat([df5_7, df5_8], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
df5_7

数据集: Total dataset has 336211 samples, and 13 features.


,Close,%b,%b-high,%b-low,macdhist,rsi,j-kdj,diff-kdj,sma-15,sma-50,sma-100,sma-200,udate
udate,,,,,,,,,,,,,
2019-01-14 17:00:00,0.000000,0.138131,0.959914,0.000000,0.421368,0.0,0.420826,0.374474,0.007473,0.002671,0.033107,0.027229,2019-01-14 17:00:00
2019-01-14 17:01:00,0.000829,0.229229,0.000000,0.000000,0.408547,0.0,0.420826,0.319732,0.006733,0.002214,0.032302,0.026722,2019-01-14 17:01:00
2019-01-14 17:02:00,0.001658,0.283814,0.000000,0.959777,0.400855,0.0,0.420826,0.278038,0.006026,0.001335,0.031460,0.026254,2019-01-14 17:02:00
2019-01-14 17:03:00,0.000000,0.285066,0.000000,0.000000,0.394017,0.0,0.430506,0.232548,0.005016,0.000703,0.030509,0.025708,2019-01-14 17:03:00
2019-01-14 17:04:00,0.005804,0.406376,0.000000,0.000000,0.396581,0.0,0.405650,0.210130,0.004511,0.000808,0.029631,0.025201,2019-01-14 17:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-18 15:56:00,0.045597,0.333628,0.000000,0.000000,0.830818,0.0,0.374900,0.626155,0.003812,0.019202,0.048337,0.047991,2020-11-18 15:56:00
2020-11-18 15:57:00,0.044025,0.333732,0.000000,0.000000,0.814932,0.0,0.374900,0.617392,0.003465,0.014290,0.036372,0.035528,2020-11-18 15:57:00
2020-11-18 15:58:00,0.055031,0.445374,0.000000,0.000000,0.808578,0.0,0.374900,0.620631,0.003951,0.010420,0.024288,0.023624,2020-11-18 15:58:00


In [9]:
# 5.5.1 訓練集, 測試集, 數據集
df5_8 = df5_7.copy(deep=True)

is_raw_data = False
if is_raw_data:
    drop_list_5 = df5_8.columns.values.tolist()
    drop_list_5.remove('Close')
    drop_list_5.remove('udate')
    df5_8.drop(drop_list_5, axis=1, inplace=True)

data4 = separate_daily(df5_8, 'dict')
days3 = np.array(list(data4.keys()))

# train_set, test_set, valid_set = days3[0:-76], days3[-76:-33], days3[-33:-2] # 0921~1113
# train_set, test_set, valid_set = days3[0:-63], days3[-63:-21], days3[-21:-2] # 1012-1113
train_set, test_set, valid_set = days3[0:-110], days3[-110:-74], days3[-74:-32] # 0608~0921
x_train, y_train, date_train = np.array([]), np.array([]), []
x_test, y_test, date_test = np.array([]), np.array([]), []
x_valid, y_valid, date_valid = np.array([]), np.array([]), []
len(train_set), len(test_set), len(valid_set)

(141, 36, 42)

In [10]:
# 5.5.2 窗口步長
t_pus_no = 5
window_size = 50
is_y_label_m_to_n = True

for day, df6 in data4.items():
    df6.drop(['udate'], axis=1, inplace=True)
    # 5.5.3 窗口
    no_max = df6.shape[0]-t_pus_no
    x_data, y_data = [], []
    for i in range(window_size, no_max):
        start, end = i-window_size, i
        # y label
        if is_y_label_m_to_n:
            temp_0 = df6.iloc[end: end+t_pus_no] # t1 ~ t5
            temp_2 = df6.iloc[end-1: end+t_pus_no-1] # current time
        else:
            temp_0 = df6.iloc[end+t_pus_no: end+t_pus_no] # t5
            temp_2 = df6.iloc[end-t_pus_no-1: end-t_pus_no-1] # current time
        y_data.append(temp_0['Close'])
        # x matrix
        temp_1 = df6.iloc[start: end]
        x_data.append(temp_1)
        # date
        days4 = temp_2.index.tolist()
        if day in train_set:
            date_train.append(days4)
        elif day in test_set:
            date_test.append(days4)
        elif day in valid_set:
            date_valid.append(days4)
    x_data, y_data = np.array(x_data), np.array(y_data)
    # print(day, x_data.shape, y_data.shape)
    # 5.5.4 分集1
    if day in train_set and x_train.any() and y_train.any():
        x_train = np.concatenate((x_train, x_data), axis=0)
        y_train = np.concatenate((y_train, y_data), axis=0)
    elif day in test_set and x_test.any() and y_test.any():
        x_test = np.concatenate((x_test, x_data), axis=0)
        y_test = np.concatenate((y_test, y_data), axis=0)
    elif day in valid_set and x_valid.any() and y_valid.any():
        x_valid = np.concatenate((x_valid, x_data), axis=0)
        y_valid = np.concatenate((y_valid, y_data), axis=0)
    # 5.5.5 分集2
    if day in train_set and not x_train.any() and not y_train.any():
        x_train, y_train = x_data, y_data
    elif day in test_set and not x_test.any() and not y_test.any():
        x_test, y_test = x_data, y_data
    elif day in valid_set and not x_valid.any() and not y_valid.any():
        x_valid, y_valid = x_data, y_data

# no_batches, timesteps, no_features
print('訓練集: X_train Data: {}, Y_train Data: {}, Date_Train: {}'.format(x_train.shape, y_train.shape, np.array(date_train).shape))
print('測試集: X_Test Data: {}, Y_Test Data: {}, Date_Test: {}'.format(x_test.shape, y_test.shape, np.array(date_test).shape))
print('验证集: X_Valid Data: {}, Y_Valid Data: {}, Date_Valid: {}'.format(x_valid.shape, y_valid.shape, np.array(date_valid).shape))

訓練集: X_train Data: (177710, 50, 12), Y_train Data: (177710, 5), Date_Train: (177710, 5)
測試集: X_Test Data: (47185, 50, 12), Y_Test Data: (47185, 5), Date_Test: (47185, 5)
验证集: X_Valid Data: (54998, 50, 12), Y_Valid Data: (54998, 5), Date_Valid: (54998, 5)


In [11]:
# 6.1.1 模型参数
batch_size = 512
epochs = 6
units = 512
verbose = 1
no_batches = x_train.shape[0]
timesteps = x_train.shape[1]
no_features = x_train.shape[2]
batch_input_shape = (timesteps, no_features)

# 6.1.2 日志参数
prefix = 'nq-lstm'
cur_time = datetime.now().strftime("%Y%m%d-%H%M%S")

# 6.2 模型 
# activation=softsign/tanh
model = tf.keras.Sequential()
model.add(LSTM(units=units, recurrent_activation='sigmoid', activation='tanh', unroll=False, use_bias=True, 
               recurrent_dropout=0, return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.2))

model.add(LSTM(units=units, activation='tanh',return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=units, activation='tanh',return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.2))

model.add(LSTM(units=units, activation='tanh',return_sequences=True, input_shape=batch_input_shape))
model.add(Dropout(rate=0.1))

model.add(LSTM(units=units, activation='tanh',return_sequences=False, input_shape=batch_input_shape))
model.add(Dropout(rate=0.2))

if is_y_label_m_to_n:
    model.add(Dense(units=t_pus_no))
else:
    model.add(Dense(units=1))

opt = tf.keras.optimizers.Adam(learning_rate=0.001) # 0.001
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])
model.summary()

# 6.3.1 check point
checkpoint_dir = './training_checkpoints/'+ prefix +'-' + cur_time
os.mkdir(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

# 6.3.2 tensor board
log_dir = os.path.join('./logs/fit/'+ prefix +'-') + cur_time
os.mkdir(log_dir)
tensor_board_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# 6.4 fit model
history_model = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test), shuffle=True, verbose=verbose, callbacks=[checkpoint_callback, tensor_board_callback])

# 6.5 save model
model_path = "./saved_model/"+prefix+"-"+cur_time+".h5"
model.save(model_path)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50, 512)           1075200   
_________________________________________________________________
dropout (Dropout)            (None, 50, 512)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 512)           2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 50, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 512)           2

In [12]:
# 7.1 visualize loss
keys = list(history_model.history.keys())
training_loss = history_model.history['loss']
test_loss = history_model.history['val_loss']
epoch_count = range(1, len(training_loss) + 1)
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('model:'+prefix+'   batch_size:'+str(batch_size)+'   epochs:'+str(epochs)+'   units:'+str(units),loc ='left')
plt.tight_layout()
plt.grid()
plt.savefig('./data/img-nq/results/'+cur_time+'-loss')
plt.clf()

# 7.2 visualize accuracy
training_accuracy = history_model.history['accuracy']
test_accuracy = history_model.history['val_accuracy']
epoch_count = range(1, len(training_accuracy) + 1)
plt.plot(epoch_count, training_accuracy, 'r--')
plt.plot(epoch_count, test_accuracy, 'b-')
plt.legend(['Train', 'Test'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('model:'+prefix+'   batch_size:'+str(batch_size)+'   epochs:'+str(epochs)+'   units:'+str(units),loc ='left')
plt.tight_layout()
plt.grid()
plt.savefig('./data/img-nq/results/'+cur_time+'-accuracy')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [13]:
# 8.0 model evaluate
verbose = 1
train_score = model.evaluate(x_train, y_train, verbose=verbose)
test_score = model.evaluate(x_test, y_test, verbose=verbose)
valid_score = model.evaluate(x_valid, y_valid, verbose=verbose)

print('Train Score: %.4f MSE (%.4f RMSE)' % (train_score[0], math.sqrt(train_score[0])))
print('Test Score: %.4f MSE (%.4f RMSE)' % (test_score[0], math.sqrt(test_score[0])))
print('Validate Score: %.4f MSE (%.4f RMSE)' % (valid_score[0], math.sqrt(valid_score[0])))

1719/1719 [==============================] - 24s 14ms/step - loss: 6.2085e-04 - accuracy: 0.3833
Train Score: 0.0002 MSE (0.0143 RMSE)
Test Score: 0.0002 MSE (0.0139 RMSE)
Validate Score: 0.0006 MSE (0.0249 RMSE)


In [14]:
# 9.0 預測
predict1 = model.predict(x_valid)
if is_y_label_m_to_n:
    columns1=['t1', 't2', 't3', 't4', 't5']
else:
    columns1=['predict_close']
df7 = pd.DataFrame(predict1, columns=columns1)
date_valid2 = [v[0] for v in date_valid]
df7['udate'] = date_valid2
df7.index = df7['udate']
df7

,t1,t2,t3,t4,t5,udate
udate,,,,,,
2020-06-08 17:49:00,0.698075,0.694951,0.697406,0.692832,0.695509,2020-06-08 17:49:00
2020-06-08 17:50:00,0.699376,0.696356,0.698906,0.694316,0.696892,2020-06-08 17:50:00
2020-06-08 17:51:00,0.700423,0.697349,0.699969,0.695396,0.697872,2020-06-08 17:51:00
2020-06-08 17:52:00,0.701034,0.697833,0.700483,0.695941,0.698349,2020-06-08 17:52:00
2020-06-08 17:53:00,0.701298,0.697966,0.700613,0.696112,0.698497,2020-06-08 17:53:00
...,...,...,...,...,...,...
2020-09-22 15:50:00,0.345701,0.344627,0.345452,0.343284,0.344198,2020-09-22 15:50:00
2020-09-22 15:51:00,0.346120,0.345191,0.346038,0.343821,0.344734,2020-09-22 15:51:00
2020-09-22 15:52:00,0.347392,0.346564,0.347446,0.345192,0.346083,2020-09-22 15:52:00


In [15]:
# 10.0 
len_shape_y = df5_7.shape[1]-2
fill_list = list(repeat(0, len_shape_y))
df8 = pd.DataFrame()

# 10.1 逆向
for k, v in scalers.items():
    mask = ((df7['udate'] >= v['day_start']) & (df7['udate'] <= v['day_end']))
    df8_1 = df7.loc[mask]
    df8_2 = df8_1.drop(['udate'], axis=1)
    # 10.2
    data4 = []
    for k1, v1 in df8_2.iterrows():
        data4_1 = []
        for v2 in v1:
            data4_1.append([v2] + fill_list)
        data4_2 = v['scaler'].inverse_transform(data4_1)
        data4_3 = [v[0] for v in data4_2]
        data4.append(data4_3)
    # 10.3
    df8_3 = pd.DataFrame(data4, columns=columns1)
    df8_3.index = df8_1['udate']
    # 10.4 合併
    if df8_3.shape[0] > 0:
        df8 = pd.concat([df8, df8_3], axis=0, join='outer', ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)

df8['udate'] = df8.index.values
df8 = df8[['udate'] + columns1]
# df8[df8.index.duplicated()]
df8

,udate,t1,t2,t3,t4,t5
udate,,,,,,
2020-06-08 17:49:00,2020-06-08 17:49:00,9847.896840,9844.890565,9847.253499,9842.851080,9845.427084
2020-06-08 17:50:00,2020-06-08 17:50:00,9849.149789,9846.242362,9848.696972,9844.279580,9846.758228
2020-06-08 17:51:00,2020-06-08 17:51:00,9850.157082,9847.198424,9849.720214,9845.318656,9847.702071
2020-06-08 17:52:00,2020-06-08 17:52:00,9850.745234,9847.664723,9850.215083,9845.843587,9848.161371
2020-06-08 17:53:00,2020-06-08 17:53:00,9850.999668,9847.792313,9850.339976,9846.008065,9848.303819
...,...,...,...,...,...,...
2020-09-22 15:50:00,2020-09-22 15:50:00,11145.732331,11144.619968,11145.474770,11143.228118,11144.175227
2020-09-22 15:51:00,2020-09-22 15:51:00,11146.166603,11145.204145,11146.081953,11143.784932,11144.730127
2020-09-22 15:52:00,2020-09-22 15:52:00,11147.484767,11146.626816,11147.541344,11145.205164,11146.128432


In [16]:
# 11.1
df9 = df3.drop(list(df3.columns)[6:], axis=1)
df9_1 = pd.concat([df9, df8], axis=1)

# 11.2 
start2, end2 =  date_valid2[0], date_valid2[-2:-1][0]
start3 = start2 - timedelta(minutes=t_pus_no+window_size)
mask3 = ((df9_1.index >= start3) & (df9_1.index <= start2)) # 窗口步長
if is_y_label_m_to_n:
    k9 = 't1'
else:
    k9 = 'predict_close'
mask4 = ((df9_1.index >= start2) & (df9_1.index <= end2)) # 預測
df9_2 = df9_1.loc[(mask3 | mask4)]
df9_3 = df9_2.drop(['udate'], axis=1)

path_name = ''
if is_raw_data:
    path_name = '-raw'
    
if is_y_label_m_to_n:
    path_name_2 = 't'+str(t_pus_no)
else:
    path_name_2 = str(t_pus_no)+'mins'

# 11.3.1 save1
mask5 = ((df9_3.index >= start3) & (df9_3.index <= start2)) # 窗口步長
mask6 = ((df9_3.index >= start2) & (df9_3.index <= end2) & (df9_3[k9] > 0)) # 預測
df9_4 = df9_3.loc[(mask5 | mask6)]

start9, end9 = pd.to_datetime(df9_4.index.values[0]), pd.to_datetime(df9_4.index.values[-1])
path_1_date = str(start9.month).zfill(2)+str(start9.day).zfill(2)+'-'+str(end9.month).zfill(2)+str(end9.day).zfill(2)
path_1 = os.path.abspath(os.path.join('data', 'nq', 'prediction', path_name_2+'-'+path_1_date+'-e'+ str(epochs) +'-u'+ str(units) + '-w' + str(window_size) + path_name+ '.csv'))
if os.path.exists(path_1):
    os.remove(path_1)
df9_4.to_csv(path_1)

# 11.3.2 save3
path_3 = os.path.abspath(os.path.join('data', 'nq', 'prediction', 'nq-prediction.csv'))
if os.path.exists(path_3):
    os.remove(path_3)
df9_4.to_csv(path_3)

# 11.4 save2
days9 = [[datetime(2020, 7, 6, 17, 0, 0), datetime(2020, 7, 10, 16, 0, 0)],
         [datetime(2020, 7, 13, 17, 0, 0), datetime(2020, 7, 17, 16, 0, 0)],
         [datetime(2020, 7, 20, 17, 0, 0), datetime(2020, 7, 24, 16, 0, 0)],
         [datetime(2020, 7, 27, 17, 0, 0), datetime(2020, 7, 31, 16, 0, 0)],
         #
         [datetime(2020, 8, 31, 17, 0, 0), datetime(2020, 9, 4, 16, 0, 0)],
         [datetime(2020, 8, 7, 17, 0, 0), datetime(2020, 9, 11, 16, 0, 0)],
         [datetime(2020, 8, 14, 17, 0, 0), datetime(2020, 9, 18, 16, 0, 0)],
         [datetime(2020, 9, 21, 17, 0, 0), datetime(2020, 9, 25, 16, 0, 0)],
         [datetime(2020, 9, 28, 17, 0, 0), datetime(2020, 10, 2, 16, 0, 0)],
         #
         [datetime(2020, 10, 5, 17, 0, 0), datetime(2020, 10, 9, 16, 0, 0)],
         [datetime(2020, 10, 12, 17, 0, 0), datetime(2020, 10, 16, 16, 0, 0)],
         [datetime(2020, 10, 19, 17, 0, 0), datetime(2020, 10, 23, 16, 0, 0)],
         [datetime(2020, 10, 26, 17, 0, 0), datetime(2020, 10, 30, 16, 0, 0)],
         #
         [datetime(2020, 11, 2, 17, 0, 0), datetime(2020, 11, 6, 16, 0, 0)],
         [datetime(2020, 11, 9, 17, 0, 0), datetime(2020, 11, 13, 16, 0, 0)]]
for v9 in days9:
    df9_5 = df9_3.copy(deep=True)
    start4 = v9[0] + timedelta(minutes=t_pus_no+window_size) # 窗口步長
    mask7 = ((df9_5.index >= v9[0]) & (df9_5.index <= start4)) # 窗口步長
    mask8 = ((df9_5.index >= start4) & (df9_5.index <= v9[1]) & (df9_5[k9] > 0)) # 預測
    df9_6 = df9_5.loc[mask7 | mask8]
    file_name_1 = path_name_2+'-'+ str(v9[0].month).zfill(2) + str(v9[0].day).zfill(2) +'-'+ str(v9[1].month).zfill(2) + str(v9[1].day).zfill(2) + '-e' + str(epochs) +'-u'+ str(units) + '-w' + str(window_size) + path_name + '.csv'
    path_2 = os.path.abspath(os.path.join('data', 'nq', 'prediction', file_name_1))
    if os.path.exists(path_2):
        os.remove(path_2)
    if df9_6.shape[0] > 1:
        df9_6.to_csv(path_2)
        print(file_name_1)

df9_3

t5-0727-0731-e6-u512-w50.csv
t5-0831-0904-e6-u512-w50.csv
t5-0807-0911-e6-u512-w50.csv
t5-0814-0918-e6-u512-w50.csv
t5-0921-0925-e6-u512-w50.csv


,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5
udate,,,,,,,,,,
2020-06-08 17:00:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:01:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:02:00,9860.25,9860.50,9859.25,9859.50,5.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:03:00,9859.50,9860.00,9857.50,9857.50,12.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:04:00,9857.50,9859.25,9857.50,9859.25,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2020-09-22 15:49:00,11149.75,11150.50,11149.50,11149.75,23.0,11146.350355,11145.134504,11145.994092,11143.785056,11144.723086
2020-09-22 15:50:00,11149.75,11151.75,11148.50,11151.75,77.0,11145.732331,11144.619968,11145.474770,11143.228118,11144.175227
2020-09-22 15:51:00,11151.75,11152.50,11150.75,11151.25,43.0,11146.166603,11145.204145,11146.081953,11143.784932,11144.730127


In [17]:
# 12.1 save mixmax scaler
path_name_12 = os.path.abspath(os.path.join('min_max_scaler', prefix+'-'+cur_time+'.pkl'))
if os.path.exists(path_name_12):
    os.remove(path_name_12)
joblib.dump(scalers, path_name_12) 

['/home/user/PycharmProjects/JupyterLab01/min_max_scaler/nq-lstm-20201130-053021.pkl']

In [18]:
# 13.1 evaluate
if is_y_label_m_to_n:
    predict_k12 = 't1'
else:
    predict_k12 = 'predict_close'

df12 = df9_3.copy(deep=True)
mask12 = ((df12[predict_k12] > 0) & (df12['Close'] > 0))
df12 = df12.loc[mask12].round(2)

# 
stats_1 = stats.describe(df12['Close'].values)
stats_2 = stats.describe(df12['t5'].values)
# R平方, 均方误差, 均方根误差, 平均绝对误差, 平均絕對百分比誤差
r2 = r2_score(df12['Close'].values, df12[predict_k12].values)
mse = mean_squared_error(df12['Close'].values, df12[predict_k12].values)
rmse = np.sqrt(mean_squared_error(df12['Close'].values, df12[predict_k12].values))
mae =  mean_absolute_error(df12['Close'].values, df12[predict_k12].values)
ampe = np.mean(np.abs((df12['Close']-df12[predict_k12])/df12['Close']))
columns2 = ['r2', 'mse', 'rmse', 'mae', 'ampe']
result1 = [r2, mse, rmse, mae, ampe]
# 最少值, 四分位數, 最大值
columns3 = ['min', 'q25%', 'q50%', 'q75%', 'max']
result2 = df12['Close'].quantile([.0, .25, .5, .75, 1]).values.tolist()
result3 = df12[predict_k12].quantile([.0, .25, .5, .75, 1]).values.tolist()

# 合併
df12_1 = pd.DataFrame([list(stats_1)+result1+result2, list(stats_2)+result1+result3], columns=list(stats_1._fields)+columns2+columns3, index=['real', 'predict'])
df12_1.drop(['minmax'], axis=1, inplace=True)

# 13.4 save
path_13 = os.path.abspath(os.path.join('data', 'nq', 'result-describe', path_name_2+'-'+path_1_date+'-e'+ str(epochs) +'-u'+ str(units) + '-w' + str(window_size) + path_name+ '-' + cur_time +'.csv'))
df12_1.to_csv(path_13)
df12_1

,nobs,mean,variance,skewness,kurtosis,r2,mse,rmse,mae,ampe,min,q25%,q50%,q75%,max
real,54997,10964.061772,551837.541409,-0.153347,-0.989223,0.999368,348.655132,18.672309,12.928292,0.001172,9490.25,10083.25,11110.25,11419.75,12444.75
predict,54997,10966.971336,555623.443490,-0.182322,-1.001850,0.999368,348.655132,18.672309,12.928292,0.001172,9500.00,10078.06,11134.24,11420.80,12452.71
